# Import dependencies

In [1]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [2]:
import time
import gc

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel
from transformers import logging

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from semeval_reader import SemevalReader

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from torch import cuda

from sa_models.SA_BERT_Dropout_BiLSTM_Linear import SA_BERT_Dropout_BiLSTM_Linear

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'

print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [4]:
logging.set_verbosity_error()

In [5]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [6]:
def get_target_list_for_polarity(polarity):
    if polarity == 'positive':
        return [0, 0, 1]
    if polarity == 'negative':
        return [1, 0, 0]
    return [0, 1, 0]

In [7]:
semeval_reader = SemevalReader('../../../data/semeval16_restaurants_train.xml')

reviews = semeval_reader.read_reviews()
absolute_polarity_sentences = semeval_reader.get_absolute_polarity_sentences()

df = pd.DataFrame(map(lambda x: (x.text, x.opinions[0].polarity), absolute_polarity_sentences))
df.rename(columns={0: 'text'}, inplace=True)
df['target_list'] = df.apply(lambda row: get_target_list_for_polarity(row[1]), axis=1)

absolute_polarity_df = df.drop(columns=[1])

# Train & Validate

In [8]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [9]:
MODEL_OUTPUT = '../../../results/SA/SemEval16 - Task 5 - Restaurants/models/bert_pre_trained_dropout_bilstm_linear.pth'
STATS_OUTPUT = '../../../results/SA/SemEval16 - Task 5 - Restaurants/stats/bert_pre_trained_dropout_bilstm_linear.csv'

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [11]:
best_accuracy = 0.0

In [12]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets)
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()

In [13]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs = model(ids, mask, token_type_ids)
            
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    return fin_outputs, fin_targets

In [14]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [15]:
for i in range(10):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/10")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True
    )

    model = SA_BERT_Dropout_BiLSTM_Linear(BertModel.from_pretrained('bert-base-uncased'), dropout=0.3, bilstm_in_features=256, no_out_labels=3, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.BCEWithLogitsLoss()

    for epoch in range(EPOCHS):
        train(epoch, model, loss_fn, optimizer, train_dataloader)

    outputs, targets = validation(model, validation_dataloader)
    outputs = np.argmax(outputs, axis=1)
    targets = np.argmax(targets, axis=1)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/313, Loss: 0.6766093969345093
Epoch: 0/2, Batch: 31/313, Loss: 0.6181526184082031
Epoch: 0/2, Batch: 62/313, Loss: 0.5439282059669495
Epoch: 0/2, Batch: 93/313, Loss: 0.499418705701828
Epoch: 0/2, Batch: 124/313, Loss: 0.5090994238853455
Epoch: 0/2, Batch: 155/313, Loss: 0.47047585248947144
Epoch: 0/2, Batch: 186/313, Loss: 0.42927083373069763
Epoch: 0/2, Batch: 217/313, Loss: 0.4422508180141449
Epoch: 0/2, Batch: 248/313, Loss: 0.3244519531726837
Epoch: 0/2, Batch: 279/313, Loss: 0.5217041969299316
Epoch: 0/2, Batch: 310/313, Loss: 0.45684099197387695
Epoch: 1/2, Batch: 0/313, Loss: 0.24235698580741882
Epoch: 1/2, Batch: 31/313, Loss: 0.2537967562675476
Epoch: 1/2, Batch: 62/313, Loss: 0.32347461581230164
Epoch: 1/2, Batch: 93/313, Loss: 0.2571392059326172
Epoch: 1/2, Batch: 124/313, Loss: 0.2059573531150818
Epoch: 1/2, Batch: 155/313, Loss: 0.15641504526138306
Epoch: 1/2, Batch: 186/313, Loss: 0.18367987871170044
Epoch: 1/2, Batch: 217/313, Loss: 0.34870

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 2/10
Epoch: 0/2, Batch: 0/313, Loss: 0.6962666511535645
Epoch: 0/2, Batch: 31/313, Loss: 0.6421117782592773
Epoch: 0/2, Batch: 62/313, Loss: 0.628750205039978
Epoch: 0/2, Batch: 93/313, Loss: 0.4942096471786499
Epoch: 0/2, Batch: 124/313, Loss: 0.49375972151756287
Epoch: 0/2, Batch: 155/313, Loss: 0.5342152118682861
Epoch: 0/2, Batch: 186/313, Loss: 0.5561062693595886
Epoch: 0/2, Batch: 217/313, Loss: 0.5179049968719482
Epoch: 0/2, Batch: 248/313, Loss: 0.4065208435058594
Epoch: 0/2, Batch: 279/313, Loss: 0.34878310561180115
Epoch: 0/2, Batch: 310/313, Loss: 0.42465972900390625
Epoch: 1/2, Batch: 0/313, Loss: 0.2688371539115906
Epoch: 1/2, Batch: 31/313, Loss: 0.3331214189529419
Epoch: 1/2, Batch: 62/313, Loss: 0.28373414278030396
Epoch: 1/2, Batch: 93/313, Loss: 0.26557356119155884
Epoch: 1/2, Batch: 124/313, Loss: 0.260492205619812
Epoch: 1/2, Batch: 155/313, Loss: 0.22669899463653564
Epoch: 1/2, Batch: 186/313, Loss: 0.16619567573070526
Epoch: 1/2, Batch: 217/313, Loss: 0.162790

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 3/10
Epoch: 0/2, Batch: 0/313, Loss: 0.6812364459037781
Epoch: 0/2, Batch: 31/313, Loss: 0.6337673664093018
Epoch: 0/2, Batch: 62/313, Loss: 0.6103806495666504
Epoch: 0/2, Batch: 93/313, Loss: 0.6215177774429321
Epoch: 0/2, Batch: 124/313, Loss: 0.47876280546188354
Epoch: 0/2, Batch: 155/313, Loss: 0.5232995748519897
Epoch: 0/2, Batch: 186/313, Loss: 0.43057364225387573
Epoch: 0/2, Batch: 217/313, Loss: 0.5643829703330994
Epoch: 0/2, Batch: 248/313, Loss: 0.30016010999679565
Epoch: 0/2, Batch: 279/313, Loss: 0.27463290095329285
Epoch: 0/2, Batch: 310/313, Loss: 0.25822287797927856
Epoch: 1/2, Batch: 0/313, Loss: 0.3732229173183441
Epoch: 1/2, Batch: 31/313, Loss: 0.42346879839897156
Epoch: 1/2, Batch: 62/313, Loss: 0.23695111274719238
Epoch: 1/2, Batch: 93/313, Loss: 0.21610236167907715
Epoch: 1/2, Batch: 124/313, Loss: 0.17269623279571533
Epoch: 1/2, Batch: 155/313, Loss: 0.17684553563594818
Epoch: 1/2, Batch: 186/313, Loss: 0.17620298266410828
Epoch: 1/2, Batch: 217/313, Loss: 0.

...

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 9/10
Epoch: 0/2, Batch: 0/313, Loss: 0.6908597946166992
Epoch: 0/2, Batch: 31/313, Loss: 0.610791802406311
Epoch: 0/2, Batch: 62/313, Loss: 0.5751376152038574
Epoch: 0/2, Batch: 93/313, Loss: 0.5629738569259644
Epoch: 0/2, Batch: 124/313, Loss: 0.508280873298645
Epoch: 0/2, Batch: 155/313, Loss: 0.4070553779602051
Epoch: 0/2, Batch: 186/313, Loss: 0.47413790225982666
Epoch: 0/2, Batch: 217/313, Loss: 0.4562471807003021
Epoch: 0/2, Batch: 248/313, Loss: 0.4552583396434784
Epoch: 0/2, Batch: 279/313, Loss: 0.3767053484916687
Epoch: 0/2, Batch: 310/313, Loss: 0.2926405668258667
Epoch: 1/2, Batch: 0/313, Loss: 0.4591086506843567
Epoch: 1/2, Batch: 31/313, Loss: 0.26777899265289307
Epoch: 1/2, Batch: 62/313, Loss: 0.4032762050628662
Epoch: 1/2, Batch: 93/313, Loss: 0.21478164196014404
Epoch: 1/2, Batch: 124/313, Loss: 0.19704803824424744
Epoch: 1/2, Batch: 155/313, Loss: 0.2219073623418808
Epoch: 1/2, Batch: 186/313, Loss: 0.2956368923187256
Epoch: 1/2, Batch: 217/313, Loss: 0.202407479

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 10/10
Epoch: 0/2, Batch: 0/313, Loss: 0.7167520523071289
Epoch: 0/2, Batch: 31/313, Loss: 0.6516823172569275
Epoch: 0/2, Batch: 62/313, Loss: 0.6124970316886902
Epoch: 0/2, Batch: 93/313, Loss: 0.5398393869400024
Epoch: 0/2, Batch: 124/313, Loss: 0.646155834197998
Epoch: 0/2, Batch: 155/313, Loss: 0.6376091837882996
Epoch: 0/2, Batch: 186/313, Loss: 0.4164033830165863
Epoch: 0/2, Batch: 217/313, Loss: 0.42600923776626587
Epoch: 0/2, Batch: 248/313, Loss: 0.3083032965660095
Epoch: 0/2, Batch: 279/313, Loss: 0.40330690145492554
Epoch: 0/2, Batch: 310/313, Loss: 0.3491010069847107
Epoch: 1/2, Batch: 0/313, Loss: 0.318656861782074
Epoch: 1/2, Batch: 31/313, Loss: 0.2974243462085724
Epoch: 1/2, Batch: 62/313, Loss: 0.456065833568573
Epoch: 1/2, Batch: 93/313, Loss: 0.21270081400871277
Epoch: 1/2, Batch: 124/313, Loss: 0.19949868321418762
Epoch: 1/2, Batch: 155/313, Loss: 0.37654945254325867
Epoch: 1/2, Batch: 186/313, Loss: 0.155507430434227
Epoch: 1/2, Batch: 217/313, Loss: 0.229827731

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.916667,0.916667,0.603548,0.916667,0.626258,0.916667,0.614611,206.506755
1,0.897436,0.897436,0.588176,0.897436,0.608921,0.897436,0.598367,201.170440
2,0.887821,0.887821,0.571667,0.887821,0.622462,0.887821,0.592196,196.178830
3,0.903846,0.903846,0.593750,0.903846,0.614777,0.903846,0.604079,196.710275
4,0.919872,0.919872,0.608766,0.919872,0.614886,0.919872,0.611542,196.209569
5,0.907051,0.907051,0.587191,0.907051,0.624424,0.907051,0.604516,196.677255
6,0.897436,0.897436,0.594422,0.897436,0.597601,0.897436,0.595719,199.181090
7,0.891026,0.891026,0.580766,0.891026,0.615034,0.891026,0.596964,202.394194
8,0.919872,0.919872,0.602778,0.919872,0.607485,0.919872,0.604803,197.855954
9,0.871795,0.871795,0.570139,0.871795,0.606928,0.871795,0.586858,196.725382


In [17]:
results.to_csv(STATS_OUTPUT)